Installing Dependencies

In [1]:
pip install -U bitsandbytes peft trl accelerate datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GemmaTokenizer,BitsAndBytesConfig

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [4]:
model_id="codellama/CodeLlama-7b-hf"

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [10]:
llm_model=AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
llm_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096

In [12]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [24]:
def formatting_func1(example):
    text = f"Structured input: {example['input'][0]}\n Code : {example['output'][0]}"
    return text

Finetuning 1 with HTML data

In [15]:
pip install pandas

In [16]:
import pandas as pd
HTML_dataframe=pd.read_csv('/content/Basic HTML Camped UI components.csv')

In [17]:
HTML_dataframe

,input,output
0,"{""title"": ""Course Progress"", ""progress"": ""70%""...","<Card>\n <Typography variant=""h2"">Course Prog..."
1,"{""title"": ""FAQs"", ""accordion"": [""What is CampE...","<Card>\n <Typography variant=""h2"">FAQs</Typog..."
2,"{""title"": ""Settings"", ""description"": ""Enable n...","<Card>\n <Typography variant=""h2"">Settings</T..."
3,"{""title"": ""FAQs"", ""accordion"": [""What is CampE...","<Card>\n <Typography variant=""h2"">FAQs</Typog..."
4,"{""title"": ""Join Our Community"", ""subtitle"": ""B...","<Card>\n <Avatar src=""https://example.com/ava..."
...,...,...
195,title: Boost Your Skills\ndescription: Empower...,"<Card>\n <Typography variant=""h2"">Boost Your ..."
196,title: Boost Your Skills\ndescription: Empower...,"<Card>\n <Typography variant=""h2"">Boost Your ..."
197,title: Join the Learning Revolution\ndescripti...,"<Card>\n <Typography variant=""h2"">Join the Le..."
198,title: Boost Your Skills\ndescription: Your pe...,"<Card>\n <Typography variant=""h2"">Boost Your ..."


In [22]:
from datasets import Dataset

hf_dataset1 = Dataset.from_pandas(HTML_dataframe)


In [25]:
trainer = SFTTrainer(
    model=llm_model,
    train_dataset=hf_dataset1,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func1,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/200 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [29]:
import wandb
wandb.init(project="my-finetuning-project")
os.environ["WANDB_DISABLED"] = "false"

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: syedmazharhussain2607 (syedmazharhussain2607-iiitdm-kancheepuram) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [30]:
trainer.train()

Step,Training Loss
1,6.091000
2,5.826900
3,5.982300
4,5.657500
5,5.423700
6,4.623500
7,4.521300
8,4.032200
9,3.742200
10,3.133000


TrainOutput(global_step=100, training_loss=1.2261944288015365, metrics={'train_runtime': 313.9643, 'train_samples_per_second': 1.274, 'train_steps_per_second': 0.319, 'total_flos': 174963297484800.0, 'train_loss': 1.2261944288015365})

Finetuning for second time

In [83]:
def formatting_func2(example):
    text = f"Component name: {example['Name'][0]}\n Code : {example['Code '][0]}"
    return text

In [89]:
component_dataframe=pd.read_csv('/content/final_components.csv',encoding='ISO-8859-1')

In [90]:
component_dataframe

,Unnamed: 0,Unnamed: 1,Name,Code
0,NaN,NaN,Accordion,"import {\n Accordion,\n AccordionContent,\n ..."
1,NaN,NaN,Alert,"import { Terminal } from ""lucide-react""\n\nimp..."
2,NaN,NaN,Aspect ratio,"import { AspectRatio } from ""@camped-ui/aspect..."
3,NaN,NaN,Avatar,"import {\n Avatar,\n AvatarFallback,\n Avat..."
4,NaN,NaN,Bento Grid,"import { BentoGrid, BentoGridItem } from ""@cam..."
5,NaN,NaN,Breadcrumb,"import {\n Breadcrumb,\n BreadcrumbEllipsis,..."
6,NaN,NaN,Button,"import { Button } from ""@camped-ui/button""\n\n..."
7,NaN,NaN,Card,"import * as React from ""react""\n\nimport { But..."
8,NaN,NaN,Carousel,"import { Card, CardContent } from ""@camped-ui/..."
9,NaN,NaN,NaN,NaN


In [91]:
component_dataframe = component_dataframe.drop(["Unnamed: 0", "Unnamed: 1"], axis=1)
component_dataframe=component_dataframe.dropna()

In [92]:
component_dataframe

,Name,Code
0,Accordion,"import {\n Accordion,\n AccordionContent,\n ..."
1,Alert,"import { Terminal } from ""lucide-react""\n\nimp..."
2,Aspect ratio,"import { AspectRatio } from ""@camped-ui/aspect..."
3,Avatar,"import {\n Avatar,\n AvatarFallback,\n Avat..."
4,Bento Grid,"import { BentoGrid, BentoGridItem } from ""@cam..."
5,Breadcrumb,"import {\n Breadcrumb,\n BreadcrumbEllipsis,..."
6,Button,"import { Button } from ""@camped-ui/button""\n\n..."
7,Card,"import * as React from ""react""\n\nimport { But..."
8,Carousel,"import { Card, CardContent } from ""@camped-ui/..."
10,Command,"import {\n Calculator,\n Calendar,\n Credit..."


In [93]:
hf_dataset2 = Dataset.from_pandas(component_dataframe)

In [94]:
trainer1 = SFTTrainer(
    model=llm_model,
    train_dataset=hf_dataset2,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func2,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/35 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [95]:
trainer1.train()

Step,Training Loss
1,6.407800
2,6.466000
3,6.280000
4,5.812100
5,4.999600
6,4.503700
7,3.644500
8,3.904400
9,3.575900
10,3.035900


TrainOutput(global_step=100, training_loss=1.3928865110874176, metrics={'train_runtime': 304.9632, 'train_samples_per_second': 1.312, 'train_steps_per_second': 0.328, 'total_flos': 154683460730880.0, 'train_loss': 1.3928865110874176})